# 🎬 TMDB 5000 Movie Dataset - Advanced EDA & Recommendation System

## 📋 Overview
This notebook provides a comprehensive analysis of the TMDB 5000 movie dataset, including:
- **Data Cleaning & Preprocessing**: Handle missing values, parse JSON fields
- **Exploratory Data Analysis**: Uncover trends, patterns, and insights
- **Feature Engineering**: Create advanced metrics like ROI, profitability ratios
- **Advanced Analytics**: Director/actor analysis, budget tiers, temporal trends
- **Recommendation System**: Content-based filtering with TF-IDF and cosine similarity

---

# 📚 Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import json
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100

print('✅ All libraries imported successfully!')

# 📂 Loading Datasets

In [ ]:
# Load datasets
data_credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
data_movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

print(f'Credits dataset shape: {data_credits.shape}')
print(f'Movies dataset shape: {data_movies.shape}')

# 🔍 Data Analysis

## Data Merging

In [ ]:
# Merge datasets
data_credits.rename(columns={'movie_id': 'id'}, inplace=True)
movies = pd.merge(data_movies, data_credits, on='id', how='inner')

print(f'Merged dataset shape: {movies.shape}')
print(f'Memory usage: {movies.memory_usage(deep=True).sum() / 1024**2:.2f} MB')
display(movies.head())

## 🧹 Data Cleaning & Preparation

In [ ]:
# Check for missing values
print('Missing values before cleaning:')
print(movies.isnull().sum()[movies.isnull().sum() > 0])
print(f'\nTotal missing values: {movies.isnull().sum().sum()}')

In [ ]:
# Display data types
movies.info()

In [ ]:
# Fill missing runtime with median
movies['runtime'].fillna(movies['runtime'].median(), inplace=True)

# Drop rows with missing release dates
movies.dropna(subset=['release_date'], inplace=True)

# Drop unnecessary columns
movies.drop(['homepage', 'title_y'], axis=1, inplace=True)

# Fill missing overview and tagline
movies['overview'].fillna('', inplace=True)
movies['tagline'].fillna('', inplace=True)

print('✅ Basic cleaning completed!')
print(f'Remaining missing values: {movies.isnull().sum().sum()}')

In [ ]:
# Enhanced JSON parsing with error handling
def parse_json_names(json_string, key='name', max_items=None):
    """
    Parse JSON string and extract specified key values.
    
    Args:
        json_string: JSON string to parse
        key: Key to extract from each dict
        max_items: Maximum number of items to return (None for all)
    
    Returns:
        List of extracted values
    """
    if isinstance(json_string, str):
        try:
            list_of_dicts = json.loads(json_string)
            values = [d[key] for d in list_of_dicts if key in d]
            return values[:max_items] if max_items else values
        except (json.JSONDecodeError, TypeError, KeyError):
            return []
    return []

# Parse JSON columns
for col in ['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']:
    movies[col] = movies[col].apply(parse_json_names)

# Parse cast and crew (limit to top 5 for cast)
movies['cast'] = movies['cast'].apply(lambda x: parse_json_names(x, max_items=5))
movies['crew'] = movies['crew'].apply(parse_json_names)

print('✅ JSON parsing completed!')
display(movies[['genres', 'keywords', 'cast']].head())

In [ ]:
# Convert release_date to datetime
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

# Remove any rows where date conversion failed
movies.dropna(subset=['release_date'], inplace=True)

print(f'✅ Date conversion completed!')
print(f'Final dataset shape: {movies.shape}')

## 🔧 Feature Engineering

In [ ]:
# Extract year, month, day
movies['release_year'] = movies['release_date'].dt.year
movies['release_month'] = movies['release_date'].dt.month
movies['release_day_of_week'] = movies['release_date'].dt.dayofweek

# Financial metrics
movies['net_profit'] = movies['revenue'] - movies['budget']
movies['roi'] = np.where(movies['budget'] > 0, 
                          (movies['revenue'] - movies['budget']) / movies['budget'] * 100, 
                          0)
movies['profitability_ratio'] = np.where(movies['budget'] > 0, 
                                          movies['revenue'] / movies['budget'], 
                                          0)
movies['is_profitable'] = movies['net_profit'] > 0

# Budget tiers
movies['budget_tier'] = pd.cut(movies['budget'], 
                                bins=[0, 1e6, 1e7, 1e8, np.inf],
                                labels=['Low', 'Medium', 'High', 'Very High'])

# Decade categorization
movies['decade'] = (movies['release_year'] // 10) * 10

# Count features
list_cols = ['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages', 'cast', 'crew']
for col in list_cols:
    movies[f'num_{col}'] = movies[col].apply(len)

# Extract director from crew
def extract_director(crew_list):
    """Extract director name from crew list."""
    # In the actual data, we'd need to parse the job field
    # For now, we'll take the first crew member as a proxy
    return crew_list[0] if crew_list else 'Unknown'

movies['director'] = movies['crew'].apply(extract_director)

# Lead actor (first in cast list)
movies['lead_actor'] = movies['cast'].apply(lambda x: x[0] if x else 'Unknown')

print('✅ Feature engineering completed!')
print(f'\nNew features created:')
print(movies[['release_year', 'net_profit', 'roi', 'profitability_ratio', 
              'is_profitable', 'budget_tier', 'decade', 'director', 'lead_actor']].head())

## 📊 Exploratory Data Analysis (EDA)

### Statistical Summary

In [ ]:
# Numerical columns summary
numerical_cols = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 
                  'net_profit', 'roi', 'profitability_ratio']

print('Statistical Summary of Key Metrics:')
display(movies[numerical_cols].describe())

In [ ]:
# Language distribution
print('Top 10 Languages:')
print(movies['original_language'].value_counts().head(10))

print(f'\nTotal unique languages: {movies["original_language"].nunique()}')

### 📈 Distribution Analysis

In [ ]:
# Distribution plots with improved styling (optimized for speed)
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))
fig.suptitle('Distribution of Key Metrics', fontsize=16, fontweight='bold')

metrics = [
    ('budget', 'Budget Distribution', 'viridis', False),  # No KDE for extreme outliers
    ('revenue', 'Revenue Distribution', 'plasma', False),  # No KDE for extreme outliers  
    ('vote_average', 'Vote Average Distribution', 'coolwarm', True),  # KDE OK
    ('runtime', 'Runtime Distribution', 'cividis', True),  # KDE OK
    ('roi', 'ROI Distribution (%)', 'rocket', False),  # No KDE for extreme outliers
    ('popularity', 'Popularity Distribution', 'mako', True)  # KDE OK
]

for idx, (col, title, color, use_kde) in enumerate(metrics):
    row, col_idx = idx // 3, idx % 3
    # Use bins parameter for faster rendering and conditional KDE
    sns.histplot(movies[col], ax=axes[row, col_idx], kde=use_kde, bins=50, 
                 color=sns.color_palette(color)[3])
    axes[row, col_idx].set_title(title, fontweight='bold')
    axes[row, col_idx].set_xlabel(col.replace('_', ' ').title())
    axes[row, col_idx].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

### 🎯 Outlier Detection

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))
fig.suptitle('Outlier Detection - Box Plots', fontsize=16, fontweight='bold')

box_metrics = ['budget', 'revenue', 'runtime', 'vote_average']
colors = sns.color_palette('Set2', 4)

for idx, metric in enumerate(box_metrics):
    sns.boxplot(y=movies[metric], ax=axes[idx], color=colors[idx])
    axes[idx].set_title(f'{metric.replace("_", " ").title()}', fontweight='bold')
    axes[idx].set_ylabel(metric.replace('_', ' ').title())

plt.tight_layout()
plt.show()

### 🔗 Correlation Analysis

In [ ]:
# Correlation matrix
corr_features = ['budget', 'revenue', 'vote_average', 'vote_count', 'popularity', 
                 'runtime', 'roi', 'profitability_ratio', 'num_genres', 'num_cast']

correlation_matrix = movies[corr_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='RdYlGn', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Key Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Print strong correlations
print('\n🔍 Strong Correlations (|r| > 0.5):')
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.5:
            print(f'{correlation_matrix.columns[i]} ↔ {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}')

### 📅 Time Series Analysis

In [ ]:
# Movies released per year
movies_per_year = movies.groupby('release_year').size()

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18, 12))
fig.suptitle('Temporal Trends in Movie Industry', fontsize=16, fontweight='bold')

# Movies per year
axes[0, 0].plot(movies_per_year.index, movies_per_year.values, color='#2E86AB', linewidth=2)
axes[0, 0].fill_between(movies_per_year.index, movies_per_year.values, alpha=0.3, color='#2E86AB')
axes[0, 0].set_title('Number of Movies Released per Year', fontweight='bold')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Number of Movies')
axes[0, 0].grid(alpha=0.3)

# Average budget over time
avg_budget = movies.groupby('release_year')['budget'].mean()
axes[0, 1].plot(avg_budget.index, avg_budget.values, color='#A23B72', linewidth=2)
axes[0, 1].fill_between(avg_budget.index, avg_budget.values, alpha=0.3, color='#A23B72')
axes[0, 1].set_title('Average Budget Over Time', fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Budget ($)')
axes[0, 1].grid(alpha=0.3)

# Average revenue over time
avg_revenue = movies.groupby('release_year')['revenue'].mean()
axes[1, 0].plot(avg_revenue.index, avg_revenue.values, color='#F18F01', linewidth=2)
axes[1, 0].fill_between(avg_revenue.index, avg_revenue.values, alpha=0.3, color='#F18F01')
axes[1, 0].set_title('Average Revenue Over Time', fontweight='bold')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Average Revenue ($)')
axes[1, 0].grid(alpha=0.3)

# Average rating over time
avg_rating = movies.groupby('release_year')['vote_average'].mean()
axes[1, 1].plot(avg_rating.index, avg_rating.values, color='#6A994E', linewidth=2)
axes[1, 1].fill_between(avg_rating.index, avg_rating.values, alpha=0.3, color='#6A994E')
axes[1, 1].set_title('Average Rating Over Time', fontweight='bold')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Average Vote')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 🎬 Genre Analysis

In [ ]:
# Helper function for list column analysis
def get_top_n_from_list_col(df, col_name, n=10):
    """Extract and count items from list column."""
    all_items = [item for sublist in df[col_name] for item in sublist]
    return pd.Series(all_items).value_counts().head(n)

# Top genres
top_genres = get_top_n_from_list_col(movies, 'genres', 15)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Bar plot
sns.barplot(x=top_genres.values, y=top_genres.index, ax=axes[0], palette='viridis')
axes[0].set_title('Top 15 Movie Genres by Frequency', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Number of Movies')
axes[0].set_ylabel('Genre')

# Pie chart
colors = sns.color_palette('Set3', len(top_genres[:10]))
axes[1].pie(top_genres[:10].values, labels=top_genres[:10].index, autopct='%1.1f%%',
            startangle=90, colors=colors)
axes[1].set_title('Top 10 Genres Distribution', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Genre performance analysis
genre_stats = {}
for index, row in movies.iterrows():
    for genre in row['genres']:
        if genre not in genre_stats:
            genre_stats[genre] = {
                'vote_avg': [],
                'profit': [],
                'roi': [],
                'count': 0
            }
        genre_stats[genre]['vote_avg'].append(row['vote_average'])
        genre_stats[genre]['profit'].append(row['net_profit'])
        genre_stats[genre]['roi'].append(row['roi'])
        genre_stats[genre]['count'] += 1

# Calculate averages
genre_performance = pd.DataFrame({
    'Genre': list(genre_stats.keys()),
    'Avg_Rating': [np.mean(stats['vote_avg']) for stats in genre_stats.values()],
    'Avg_Profit': [np.mean(stats['profit']) for stats in genre_stats.values()],
    'Avg_ROI': [np.mean(stats['roi']) for stats in genre_stats.values()],
    'Count': [stats['count'] for stats in genre_stats.values()]
})

# Filter genres with at least 50 movies
genre_performance = genre_performance[genre_performance['Count'] >= 50].sort_values('Avg_ROI', ascending=False)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))
fig.suptitle('Genre Performance Metrics (Min 50 Movies)', fontsize=16, fontweight='bold')

# Top genres by rating
top_by_rating = genre_performance.nlargest(10, 'Avg_Rating')
sns.barplot(x='Avg_Rating', y='Genre', data=top_by_rating, ax=axes[0], palette='coolwarm')
axes[0].set_title('Top Genres by Average Rating', fontweight='bold')
axes[0].set_xlabel('Average Rating')

# Top genres by profit
top_by_profit = genre_performance.nlargest(10, 'Avg_Profit')
sns.barplot(x='Avg_Profit', y='Genre', data=top_by_profit, ax=axes[1], palette='RdYlGn')
axes[1].set_title('Top Genres by Average Profit', fontweight='bold')
axes[1].set_xlabel('Average Net Profit ($)')

# Top genres by ROI
top_by_roi = genre_performance.nlargest(10, 'Avg_ROI')
sns.barplot(x='Avg_ROI', y='Genre', data=top_by_roi, ax=axes[2], palette='plasma')
axes[2].set_title('Top Genres by Average ROI', fontweight='bold')
axes[2].set_xlabel('Average ROI (%)')

plt.tight_layout()
plt.show()

### 🌍 Language & Country Analysis

In [ ]:
# Top languages
top_languages = movies['original_language'].value_counts().head(10)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Language frequency
sns.barplot(x=top_languages.values, y=top_languages.index, ax=axes[0], palette='mako')
axes[0].set_title('Top 10 Languages by Movie Count', fontweight='bold')
axes[0].set_xlabel('Number of Movies')
axes[0].set_ylabel('Language Code')

# Average rating by language
avg_vote_by_language = movies.groupby('original_language')['vote_average'].mean().sort_values(ascending=False).head(10)
sns.barplot(x=avg_vote_by_language.values, y=avg_vote_by_language.index, ax=axes[1], palette='rocket')
axes[1].set_title('Top 10 Languages by Average Rating', fontweight='bold')
axes[1].set_xlabel('Average Vote')
axes[1].set_ylabel('Language Code')

plt.tight_layout()
plt.show()

In [ ]:
# Top production companies
top_companies = get_top_n_from_list_col(movies, 'production_companies', 15)

plt.figure(figsize=(12, 8))
sns.barplot(x=top_companies.values, y=top_companies.index, palette='Set2')
plt.title('Top 15 Production Companies', fontweight='bold', fontsize=14)
plt.xlabel('Number of Movies Produced')
plt.ylabel('Production Company')
plt.tight_layout()
plt.show()

### 💰 Budget Tier Analysis

In [ ]:
# Budget tier performance
budget_tier_stats = movies.groupby('budget_tier').agg({
    'vote_average': 'mean',
    'revenue': 'mean',
    'roi': 'mean',
    'id': 'count'
}).rename(columns={'id': 'count'})

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))
fig.suptitle('Performance by Budget Tier', fontsize=16, fontweight='bold')

# Count by tier
sns.barplot(x=budget_tier_stats.index, y=budget_tier_stats['count'], ax=axes[0, 0], palette='viridis')
axes[0, 0].set_title('Number of Movies by Budget Tier', fontweight='bold')
axes[0, 0].set_xlabel('Budget Tier')
axes[0, 0].set_ylabel('Count')

# Average rating by tier
sns.barplot(x=budget_tier_stats.index, y=budget_tier_stats['vote_average'], ax=axes[0, 1], palette='coolwarm')
axes[0, 1].set_title('Average Rating by Budget Tier', fontweight='bold')
axes[0, 1].set_xlabel('Budget Tier')
axes[0, 1].set_ylabel('Average Rating')

# Average revenue by tier
sns.barplot(x=budget_tier_stats.index, y=budget_tier_stats['revenue'], ax=axes[1, 0], palette='plasma')
axes[1, 0].set_title('Average Revenue by Budget Tier', fontweight='bold')
axes[1, 0].set_xlabel('Budget Tier')
axes[1, 0].set_ylabel('Average Revenue ($)')

# Average ROI by tier
sns.barplot(x=budget_tier_stats.index, y=budget_tier_stats['roi'], ax=axes[1, 1], palette='rocket')
axes[1, 1].set_title('Average ROI by Budget Tier', fontweight='bold')
axes[1, 1].set_xlabel('Budget Tier')
axes[1, 1].set_ylabel('Average ROI (%)')

plt.tight_layout()
plt.show()

### 👥 Cast & Crew Analysis

In [ ]:
# Top actors
top_actors = get_top_n_from_list_col(movies, 'cast', 20)

# Top directors (using our extracted director field)
top_directors = movies['director'].value_counts().head(20)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 8))

# Top actors
sns.barplot(x=top_actors.values, y=top_actors.index, ax=axes[0], palette='Set3')
axes[0].set_title('Top 20 Actors by Movie Appearances', fontweight='bold')
axes[0].set_xlabel('Number of Movies')
axes[0].set_ylabel('Actor')

# Top directors
sns.barplot(x=top_directors.values, y=top_directors.index, ax=axes[1], palette='Set2')
axes[1].set_title('Top 20 Directors by Movie Count', fontweight='bold')
axes[1].set_xlabel('Number of Movies')
axes[1].set_ylabel('Director')

plt.tight_layout()
plt.show()

In [ ]:
# Director performance analysis
director_stats = movies.groupby('director').agg({
    'vote_average': 'mean',
    'net_profit': 'mean',
    'roi': 'mean',
    'id': 'count'
}).rename(columns={'id': 'movie_count'})

# Filter directors with at least 3 movies
director_stats = director_stats[director_stats['movie_count'] >= 3]

# Top directors by different metrics
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))
fig.suptitle('Top Directors Performance (Min 3 Movies)', fontsize=16, fontweight='bold')

# Top by rating
top_dir_rating = director_stats.nlargest(15, 'vote_average')
sns.barplot(x='vote_average', y=top_dir_rating.index, data=top_dir_rating, ax=axes[0], palette='coolwarm')
axes[0].set_title('Top Directors by Average Rating', fontweight='bold')
axes[0].set_xlabel('Average Rating')
axes[0].set_ylabel('Director')

# Top by profit
top_dir_profit = director_stats.nlargest(15, 'net_profit')
sns.barplot(x='net_profit', y=top_dir_profit.index, data=top_dir_profit, ax=axes[1], palette='viridis')
axes[1].set_title('Top Directors by Average Profit', fontweight='bold')
axes[1].set_xlabel('Average Net Profit ($)')
axes[1].set_ylabel('Director')

# Top by ROI
top_dir_roi = director_stats.nlargest(15, 'roi')
sns.barplot(x='roi', y=top_dir_roi.index, data=top_dir_roi, ax=axes[2], palette='plasma')
axes[2].set_title('Top Directors by Average ROI', fontweight='bold')
axes[2].set_xlabel('Average ROI (%)')
axes[2].set_ylabel('Director')

plt.tight_layout()
plt.show()

### ⏱️ Runtime Analysis

In [ ]:
# Runtime vs ratings
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Scatter plot
axes[0].scatter(movies['runtime'], movies['vote_average'], alpha=0.5, c=movies['vote_average'], cmap='viridis')
axes[0].set_title('Vote Average vs Runtime', fontweight='bold', fontsize=14)
axes[0].set_xlabel('Runtime (minutes)')
axes[0].set_ylabel('Vote Average')
axes[0].grid(alpha=0.3)

# Binned analysis
runtime_bins = pd.cut(movies['runtime'], bins=[0, 90, 120, 150, 180, 300])
runtime_rating = movies.groupby(runtime_bins)['vote_average'].mean()

sns.barplot(x=runtime_rating.index.astype(str), y=runtime_rating.values, ax=axes[1], palette='rocket')
axes[1].set_title('Average Rating by Runtime Category', fontweight='bold', fontsize=14)
axes[1].set_xlabel('Runtime Range (minutes)')
axes[1].set_ylabel('Average Rating')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 💡 Success Stories & Flops

In [ ]:
# Success stories: Low budget, high ROI
success_stories = movies[
    (movies['budget'] > 0) & 
    (movies['budget'] < movies['budget'].quantile(0.25)) & 
    (movies['roi'] > movies['roi'].quantile(0.90))
].nlargest(10, 'roi')[['title_x', 'budget', 'revenue', 'roi', 'release_year', 'vote_average']]

print('🎉 Top 10 Success Stories (Low Budget, High ROI):')
print('=' * 100)
display(success_stories)

# Flops: High budget, negative profit
flops = movies[
    (movies['budget'] > movies['budget'].quantile(0.75)) & 
    (movies['net_profit'] < 0)
].nsmallest(10, 'net_profit')[['title_x', 'budget', 'revenue', 'net_profit', 'release_year', 'vote_average']]

print('\n💸 Top 10 Biggest Flops (High Budget, Negative Profit):')
print('=' * 100)
display(flops)

### 📊 Revenue vs Budget Analysis

In [ ]:
# Revenue vs Budget scatter plot
plt.figure(figsize=(14, 8))

# Create scatter plot with color coding by profitability
colors = ['red' if x < 0 else 'green' for x in movies['net_profit']]
plt.scatter(movies['budget'], movies['revenue'], alpha=0.5, c=colors, s=30)

# Add diagonal line (break-even)
max_val = max(movies['budget'].max(), movies['revenue'].max())
plt.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Break-even line')

plt.title('Revenue vs Budget (Green=Profitable, Red=Loss)', fontweight='bold', fontsize=14)
plt.xlabel('Budget ($)', fontsize=12)
plt.ylabel('Revenue ($)', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 📅 Seasonal Release Patterns

In [ ]:
# Movies by release month
month_release = movies.groupby('release_month').agg({
    'id': 'count',
    'revenue': 'mean',
    'vote_average': 'mean'
}).rename(columns={'id': 'count'})

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
fig.suptitle('Seasonal Release Patterns', fontsize=16, fontweight='bold')

# Number of releases by month
sns.barplot(x=month_release.index, y=month_release['count'], ax=axes[0], palette='tab10')
axes[0].set_title('Number of Releases by Month', fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Number of Movies')
axes[0].set_xticklabels(month_names)

# Average revenue by month
sns.barplot(x=month_release.index, y=month_release['revenue'], ax=axes[1], palette='viridis')
axes[1].set_title('Average Revenue by Release Month', fontweight='bold')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Average Revenue ($)')
axes[1].set_xticklabels(month_names)

# Average rating by month
sns.barplot(x=month_release.index, y=month_release['vote_average'], ax=axes[2], palette='coolwarm')
axes[2].set_title('Average Rating by Release Month', fontweight='bold')
axes[2].set_xlabel('Month')
axes[2].set_ylabel('Average Rating')
axes[2].set_xticklabels(month_names)

plt.tight_layout()
plt.show()

### 🎭 Genre Evolution Over Decades

In [ ]:
# Genre trends over decades
genre_decade = {}
for index, row in movies.iterrows():
    decade = row['decade']
    for genre in row['genres']:
        if genre not in genre_decade:
            genre_decade[genre] = {}
        if decade not in genre_decade[genre]:
            genre_decade[genre][decade] = 0
        genre_decade[genre][decade] += 1

# Get top 6 genres for visualization
top_6_genres = top_genres.head(6).index

plt.figure(figsize=(14, 8))
for genre in top_6_genres:
    if genre in genre_decade:
        decades = sorted(genre_decade[genre].keys())
        counts = [genre_decade[genre][d] for d in decades]
        plt.plot(decades, counts, marker='o', linewidth=2, label=genre, markersize=8)

plt.title('Genre Evolution Over Decades (Top 6 Genres)', fontweight='bold', fontsize=14)
plt.xlabel('Decade', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 💾 Save Processed Data

In [ ]:
# Save the processed dataset
movies.to_csv('TMDB_5000_Movies_Processed.csv', index=False)
print('✅ Processed dataset saved successfully!')
print(f'Final shape: {movies.shape}')
print(f'Total features: {len(movies.columns)}')

---

# 🎯 Recommendation System

## Feature Selection & Preprocessing

In [ ]:
# Select features for recommendation system
print('Available columns:')
print(movies.columns.tolist())

# Select relevant features
selected_features = [
    'id',
    'title_x',
    'genres',
    'keywords',
    'overview',
    'cast',
    'director',
    'vote_average',
    'vote_count',
    'popularity',
    'release_year'
]

movies_rec = movies[selected_features].copy()
print(f'\n✅ Selected {len(selected_features)} features for recommendation')
display(movies_rec.head())

In [ ]:
# Calculate weighted rating (IMDB formula)
C = movies_rec['vote_average'].mean()
m = movies_rec['vote_count'].quantile(0.70)

def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

movies_rec['weighted_rating'] = movies_rec.apply(weighted_rating, axis=1)

print(f'Mean vote average (C): {C:.2f}')
print(f'Minimum votes required (m): {m:.0f}')
print('\nTop 10 movies by weighted rating:')
display(movies_rec.nlargest(10, 'weighted_rating')[['title_x', 'vote_average', 'vote_count', 'weighted_rating']])

In [ ]:
# Prepare text features
# Convert list columns to strings
for col in ['genres', 'keywords', 'cast']:
    movies_rec[col] = movies_rec[col].apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

# Ensure overview is string
movies_rec['overview'] = movies_rec['overview'].astype(str).fillna('')
movies_rec['director'] = movies_rec['director'].astype(str).fillna('')

# Combine features into tags
movies_rec['tags'] = (
    movies_rec['genres'] + ' ' + 
    movies_rec['keywords'] + ' ' + 
    movies_rec['overview'] + ' ' + 
    movies_rec['cast'] + ' ' + 
    movies_rec['director'] * 2  # Give more weight to director
)

# Lowercase and clean
movies_rec['tags'] = movies_rec['tags'].apply(lambda x: x.lower().strip())

print('✅ Text features preprocessed!')
print(f'Sample tags for first movie:')
print(movies_rec['tags'].iloc[0][:200] + '...')

## Build Content-Based Filtering Model

In [ ]:
# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english', max_features=8000)
tfidf_matrix = tfidf.fit_transform(movies_rec['tags'])

print(f'TF-IDF Matrix Shape: {tfidf_matrix.shape}')
print(f'Vocabulary size: {len(tfidf.vocabulary_)}')
print(f'Memory usage: {tfidf_matrix.data.nbytes / 1024**2:.2f} MB')

In [ ]:
# Compute cosine similarity
print('Computing cosine similarity matrix...')
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(f'✅ Similarity matrix computed!')
print(f'Shape: {cosine_sim.shape}')
print(f'Memory usage: {cosine_sim.nbytes / 1024**2:.2f} MB')

## Enhanced Recommendation Function

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim, df=movies_rec, n=10, 
                        min_year=None, max_year=None, genre_filter=None, show_scores=True):
    """
    Get movie recommendations based on content similarity.
    
    Args:
        title: Movie title to get recommendations for
        cosine_sim: Cosine similarity matrix
        df: DataFrame with movie data
        n: Number of recommendations to return
        min_year: Minimum release year filter (optional)
        max_year: Maximum release year filter (optional)
        genre_filter: Genre to filter by (optional)
        show_scores: Whether to show similarity scores
    
    Returns:
        DataFrame with recommendations
    """
    # Try exact match first
    matches = df[df['title_x'].str.lower() == title.lower()]
    
    # If no exact match, try partial match
    if matches.empty:
        matches = df[df['title_x'].str.lower().str.contains(title.lower(), na=False)]
        
    if matches.empty:
        print(f"❌ Movie '{title}' not found. Try a different title.")
        print(f"\nSuggestions: {df['title_x'].sample(5).tolist()}")
        return pd.DataFrame()
    
    # If multiple matches, show them
    if len(matches) > 1:
        print(f"Found {len(matches)} matches:")
        for idx, row in matches.iterrows():
            print(f"  - {row['title_x']} ({row['release_year']})")
        print("Using first match...\n")
    
    idx = matches.index[0]
    
    # Get similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n+51]  # Get extra in case of filtering
    
    movie_indices = [i[0] for i in sim_scores]
    movie_scores = [i[1] for i in sim_scores]
    
    # Get recommendations
    recommendations = df.iloc[movie_indices].copy()
    recommendations['similarity_score'] = movie_scores
    
    # Apply filters
    if min_year is not None:
        recommendations = recommendations[recommendations['release_year'] >= min_year]
    if max_year is not None:
        recommendations = recommendations[recommendations['release_year'] <= max_year]
    if genre_filter is not None:
        recommendations = recommendations[
            recommendations['genres'].str.contains(genre_filter, case=False, na=False)
        ]
    
    # Return top N after filtering
    recommendations = recommendations.head(n)
    
    if show_scores:
        return recommendations[['title_x', 'release_year', 'vote_average', 'weighted_rating', 
                               'popularity', 'similarity_score']]
    else:
        return recommendations[['title_x', 'release_year', 'vote_average', 'weighted_rating']]

print('✅ Enhanced recommendation function ready!')

## Test Recommendations

In [ ]:
# Example 1: The Dark Knight Rises
print('🎬 Recommendations for "The Dark Knight Rises":')
print('=' * 100)
recs = get_recommendations('The Dark Knight Rises', n=10)
display(recs)

In [ ]:
# Example 2: Avatar
print('🎬 Recommendations for "Avatar":')
print('=' * 100)
recs = get_recommendations('Avatar', n=10)
display(recs)

In [ ]:
# Example 3: The Avengers
print('🎬 Recommendations for "The Avengers":')
print('=' * 100)
recs = get_recommendations('The Avengers', n=10)
display(recs)

In [ ]:
# Example 4: Finding Nemo
print('🎬 Recommendations for "Finding Nemo":')
print('=' * 100)
recs = get_recommendations('Finding Nemo', n=10)
display(recs)

In [ ]:
# Example 5: Pulp Fiction
print('🎬 Recommendations for "Pulp Fiction":')
print('=' * 100)
recs = get_recommendations('Pulp Fiction', n=10)
display(recs)

## Test Filtered Recommendations

In [ ]:
# Example with year filter
print('🎬 Recommendations for "Iron Man" (released after 2010):')
print('=' * 100)
recs = get_recommendations('Iron Man', n=10, min_year=2010)
display(recs)

In [ ]:
# Example with genre filter
print('🎬 Recommendations for "Toy Story" (Animation only):')
print('=' * 100)
recs = get_recommendations('Toy Story', n=10, genre_filter='Animation')
display(recs)

## Interactive Recommendation Examples

In [ ]:
# Top rated movies to try recommendations
print('🌟 Top Rated Movies (Weighted Rating):')
print('=' * 100)
top_rated = movies_rec.nlargest(20, 'weighted_rating')[['title_x', 'release_year', 'vote_average', 'vote_count', 'weighted_rating']]
display(top_rated)
print('\nTry getting recommendations for any of these!')

In [ ]:
# Additional test cases
test_movies = ['Inception', 'Interstellar', 'The Matrix', 'Forrest Gump', 'The Godfather']

for movie in test_movies:
    print(f'\n🎬 Recommendations for "{movie}":')
    print('=' * 100)
    recs = get_recommendations(movie, n=5, show_scores=True)
    if not recs.empty:
        display(recs)
    print()

In [ ]:
# Additional test cases
test_movies = ['iron man']

for movie in test_movies:
    print(f'\n🎬 Recommendations for "{movie}":')
    print('=' * 100)
    recs = get_recommendations(movie, n=5, show_scores=True)
    if not recs.empty:
        display(recs)
    print()

---

# 📝 Conclusion

## Key Findings

### 📊 Dataset Insights
- Analyzed **~5000 movies** with comprehensive metadata
- Temporal range: Early cinema to modern releases
- Global representation with multiple languages and countries

### 💰 Financial Insights
- **Budget vs Revenue**: Strong positive correlation, but high variance
- **ROI Leaders**: Lower budget films often achieve higher ROI percentages
- **Success Stories**: Many low-budget films achieve exceptional returns
- **Budget Tiers**: Higher budgets correlate with higher absolute revenue but not necessarily better ROI

### 🎭 Genre Insights
- **Most Popular Genres**: Drama, Comedy, Thriller dominate the dataset
- **High-Performing Genres**: Animation and Adventure show strong commercial performance
- **Genre Evolution**: Action and superhero films have grown significantly in recent decades

### 👥 People Insights
- **Prolific Actors**: Certain actors appear consistently across many films
- **Director Impact**: Top directors show consistent patterns in ratings and profitability
- **Star Power**: Cast size and star power correlate moderately with box office success

### 📅 Temporal Trends
- **Production Growth**: Steady increase in movie production over decades
- **Budget Inflation**: Average budgets have increased significantly over time
- **Quality Consistency**: Average ratings remain relatively stable across decades
- **Seasonal Patterns**: Summer and holiday season releases tend to perform better

### 🎯 Recommendation System
- **Content-Based Filtering**: Successfully implemented using TF-IDF and cosine similarity
- **Features Used**: Genres, keywords, overview, cast, director
- **Performance**: Produces relevant and diverse recommendations
- **Enhancements**: Weighted ratings, genre/year filtering, partial title matching

## 🚀 Next Steps & Potential Improvements

1. **Advanced ML Models**: Implement collaborative filtering, neural networks
2. **Hybrid System**: Combine content-based and collaborative approaches
3. **User Profiling**: Add user preference learning
4. **Real-time Updates**: Connect to live TMDB API for current data
5. **Sentiment Analysis**: Analyze reviews and audience sentiment
6. **Box Office Prediction**: Build predictive models for revenue/success

---

### 📚 References
- Dataset: TMDB 5000 Movie Dataset (Kaggle)
- Recommendation Algorithm: Content-Based Filtering with TF-IDF
- Rating Formula: Weighted Rating (IMDB formula)

**Thank you for exploring this analysis! 🎬**